# Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna, åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna.

### Våra frågeställningar:
- Kolla igenom vilka sporter som varit med?
- Analysera "Tug-Of-War"! (dragkamp) Kul att tex kolla hur länge det varit med i OS, vilka har varit bäst? Osv.
- Finns det andra gamla eller intressanta sporter att titta på?
- Alpinism?
- Aeronautics [https://www.topendsports.com/events/demonstration/aeronautics.htm](https://www.topendsports.com/events/demonstration/aeronautics.htm)
- Yngsta, äldsta medaljörerna?
#### "Tug of war" (Dragkamp) (Daniel):
- Vilka länder har deltagit och hur ser medaljfördelningen ut?
- Vilka år var detta en sport i OS?

#### Kulstötning / Slägga / Spjut / Diskus (Daniel):
- Hur ser längd- och vikt-utvecklingen ut genom historien?
- Analysera män och kvinnor separat
- Hade varit intressant att jämföra med resultaten i sporterna också....
- Distribution plots? Box-plots? Eller en animation frame, som går igenom åren?